# Pileup analysis

### Initialize a PySeQuiLa session

In [ ]:
from pyspark.sql import SparkSession
from pysequila import SequilaSession
import pandas as pd
pd.options.display.max_columns = None
pd.set_option('max_colwidth', None)


spark = SparkSession \
.builder \
.getOrCreate()

ss = SequilaSession(spark)

### Download sample data

In [ ]:
%%bash
mkdir -p data
cd data
wget --quiet http://biodatageeks.ii.pw.edu.pl/sequila/data/NA12878.multichrom.md.bam
wget --quiet http://biodatageeks.ii.pw.edu.pl/sequila/data/Homo_sapiens_assembly18_chr1_chrM.small.fasta
wget --quiet http://biodatageeks.ii.pw.edu.pl/sequila/data/Homo_sapiens_assembly18_chr1_chrM.small.fasta.fai

### Set helper variables

In [ ]:
import os
base_path = f"{os.getcwd()}/data"

bam_path = f'{base_path}/NA12878.multichrom.md.bam'
ref_path=f'{base_path}/Homo_sapiens_assembly18_chr1_chrM.small.fasta'
sample_id= 'NA12878'
table_name = 'reads'

app_name = "sequila"

#### Create a table

In [ ]:
ss.sql(f'''CREATE TABLE IF NOT EXISTS {table_name} \
         USING org.biodatageeks.sequila.datasources.BAM.BAMDataSource \
         OPTIONS(path "{bam_path}")''')

## `pileup`
`pileup(tableName: String, sampleId: String, referencePath: String, includeBaseQual: Boolean = False)`

##### Compute reads pileup over using aligment data

#### input parameters
* tableName - registered table over alignment files (see File formats for details)
* sampleId - name of the sample correspondind to the name of the alignment file without a file extension
* referencePath - a local path to referenece in a FASTQ format (should be indexed and available on all computing nodes if run in distributed nodes). Can be also distributed in the runtime using `--files` parameter to `pyspark-shell`
* includeBaseQual - determines whether base qualities should be computed and inluded in the output (defaults to False). Please note that calculating base qualities has **significant** impact on performance (see Benchmark page for details).

#### returned columns
Blocks of pileup with the following columns:

* sample_id - name of the sample correspondind to the name of the alignment file without a file extension
* contig - contig name
* pos_start - start postition of a block
* pos_end - end postition of a block
* coverage - depth of coverage for a block or single position
* countRef - depth of coverage of reads that have at a give position base equal to reference
* alts - map of alts in format `(ASCII code of alt: coverage)`
* quals - map of base qualities in format (`base: Array[qualities]`)



In [ ]:
ss.sql(f'''SELECT contig, pos_start, pos_end, ref, coverage, countRef, alts \
      FROM  pileup('{table_name}', '{sample_id}', '{ref_path}', true) LIMIT 10''').toPandas()

## `to_charmap`
`to_charmap(quals: Map(Base: Short, Array[BaseQuality])`

##### Convert binary representation of base qualities into human-readable map

#### example:

In [ ]:
ss.sql(f'''SELECT quals, to_charmap(quals) AS quals_decoded \
      FROM  pileup('{table_name}', '{sample_id}', '{ref_path}', true) LIMIT 10''').toPandas()

## `to_char`
`to_char(alts: Map(Alt: Short, coverage:Short)`

##### Convert binary representation of alts into human-readable map with strand information encoded as lower/upper case

#### example:

In [ ]:
ss.sql(f'''SELECT alts, to_char(alts) AS alts_decoded \
      FROM  pileup('{table_name}', '{sample_id}', '{ref_path}', true) LIMIT 10''').toPandas()